In [ ]:

# library imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# modelling and data split 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error


In [ ]:
# reading the train data 
full_df = pd.read_csv('./data/train.csv', index_col="Id")
X_test = pd.read_csv('./data/test.csv', index_col="Id")


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


NameError: name 'full_df' is not defined

In [37]:
# set the target and features
y = full_df.SalePrice.copy()
features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = full_df[features].copy()

In [38]:
X.describe()

,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,10516.828082,1971.267808,1162.626712,346.992466,1.565068,2.866438,6.517808
std,9981.264932,30.202904,386.587738,436.528436,0.550916,0.815778,1.625393
min,1300.000000,1872.000000,334.000000,0.000000,0.000000,0.000000,2.000000
25%,7553.500000,1954.000000,882.000000,0.000000,1.000000,2.000000,5.000000
50%,9478.500000,1973.000000,1087.000000,0.000000,2.000000,3.000000,6.000000
75%,11601.500000,2000.000000,1391.250000,728.000000,2.000000,3.000000,7.000000
max,215245.000000,2010.000000,4692.000000,2065.000000,3.000000,8.000000,14.000000


In [39]:
# splitting the  data 
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=1)

In [40]:
# building the model 
# testing multipl models 
model_1 = RandomForestRegressor(n_estimators=50,random_state=1)
model_2 = RandomForestRegressor(n_estimators=100, random_state=1)
model_3 = RandomForestRegressor(n_estimators=100,criterion="mae", random_state=1)
model_4 = RandomForestRegressor(n_estimators=200, min_samples_split=20, random_state=1)
model_5 = RandomForestRegressor(n_estimators=100,max_depth=7, random_state=1)

models = [model_1, model_2, model_3, model_4,model_5]


In [41]:
def model_eval(model, X_t=X_train, X_v=X_val, y_t=y_train, y_v=y_val):
    model.fit(X_t, y_t)
    predictions = model.predict(X_val)
    return mean_absolute_error(y_v, predictions)

In [42]:
minimum_mae = 25000

maximum_mae = 0
for i, model in enumerate(models):
    mae = model_eval(model)
    print(f"MAE for model {i+1} is: \n\t {mae}")
    if mae < minimum_mae:
        minimum_mae = mae
        min_num = i + 1
        my_model = model
        
    if mae > maximum_mae:
        maximum_mae = mae
        max_num = i + 1 

print(f"The model with the minimum MAE is model {min_num}: {minimum_mae:.2f}")
print(f"The model with the maximum MAE is model {max_num}: {maximum_mae:.2f}")

MAE for model 1 is: 
	 22024.487689497717
MAE for model 2 is: 
	 21857.15912981083
MAE for model 3 is: 
	 21878.804589041094
MAE for model 4 is: 
	 22669.02789162209
MAE for model 5 is: 
	 22352.5764886615
The model with the minimum MAE is model 2: 21857.16
The model with the maximum MAE is model 4: 22669.03


In [43]:
my_model # the model with the minimum mae

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=1, verbose=0, warm_start=False)

In [50]:
preds_test = my_model.predict(X_test[features])

In [51]:
# Save predictions in format used for competition scoring
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('./data/submission.csv', index=False)